Exercises:
**E01: train a trigram language model**, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model? 


**E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set**. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?  


**E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model** - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

**E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?**


**E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?**  


**E06: meta-exercise! Think of a fun/interesting exercise and complete it.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
words = open('/content/drive/MyDrive/data science/names.txt', 'r').read().splitlines()

In [3]:
t = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    trigram = (ch1+ch2, ch3)
    t[trigram] = t.get(trigram, 0) + 1

In [4]:
sorted(t.items(), key = lambda kv: -kv[1])

[(('ah', '<E>'), 1714),
 (('na', '<E>'), 1673),
 (('an', '<E>'), 1509),
 (('on', '<E>'), 1503),
 (('<S>m', 'a'), 1453),
 (('<S>j', 'a'), 1255),
 (('<S>k', 'a'), 1254),
 (('en', '<E>'), 1217),
 (('ly', 'n'), 976),
 (('yn', '<E>'), 953),
 (('ar', 'i'), 950),
 (('ia', '<E>'), 903),
 (('ie', '<E>'), 858),
 (('an', 'n'), 825),
 (('el', 'l'), 822),
 (('an', 'a'), 804),
 (('ia', 'n'), 790),
 (('ma', 'r'), 776),
 (('in', '<E>'), 766),
 (('el', '<E>'), 727),
 (('ya', '<E>'), 716),
 (('an', 'i'), 703),
 (('<S>d', 'a'), 700),
 (('la', '<E>'), 684),
 (('er', '<E>'), 683),
 (('iy', 'a'), 669),
 (('la', 'n'), 647),
 (('<S>b', 'r'), 646),
 (('nn', 'a'), 633),
 (('<S>a', 'l'), 632),
 (('<S>c', 'a'), 628),
 (('ra', '<E>'), 627),
 (('ni', '<E>'), 625),
 (('<S>a', 'n'), 623),
 (('nn', '<E>'), 619),
 (('ne', '<E>'), 607),
 (('ee', '<E>'), 605),
 (('ey', '<E>'), 602),
 (('<S>k', 'e'), 601),
 (('al', 'e'), 601),
 (('<S>s', 'a'), 595),
 (('al', 'i'), 575),
 (('sh', 'a'), 562),
 (('el', 'i'), 537),
 (('<S>d',

In [4]:
import torch

In [5]:
N = torch.zeros((27*27, 27), dtype=torch.int32)

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [7]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1] * stoi[ch2]
    ix2 = stoi[ch3]
    N[ix1, ix2] += 1

In [ ]:
N[645,]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=torch.int32)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(500,500))
plt.imshow(N, cmap='Blues')
# for i in range(27):
#     for j in range(27):
#       for k in range(27):
#         chstr = itos[i] + itos[j] + itos[k]
#         plt.text(k, i*j, chstr, ha="center", va="bottom", color='gray')
#         plt.text(k, i*j, N[i*j, k].item(), ha="center", va="top", color='gray')
# plt.axis('off');

In [ ]:
N[0]

tensor([    0, 10075,   234,   142,   494,  4089,    44,    38,  1527,  2809,
           50,   161,  1568,   789,   807,  2599,    47,    11,  2209,   481,
          269,  1331,   472,    92,    36,  1346,   313], dtype=torch.int32)

In [9]:
p = N[0].float()
p = p/p.sum()
p

tensor([0.0000, 0.3145, 0.0073, 0.0044, 0.0154, 0.1276, 0.0014, 0.0012, 0.0477,
        0.0877, 0.0016, 0.0050, 0.0489, 0.0246, 0.0252, 0.0811, 0.0015, 0.0003,
        0.0690, 0.0150, 0.0084, 0.0416, 0.0147, 0.0029, 0.0011, 0.0420, 0.0098])

In [10]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'm'

In [11]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

tensor([0.6064, 0.3033, 0.0903])

In [12]:
torch.multinomial(p, num_samples=100, replacement=True, generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0,
        0, 1, 1, 1])

In [13]:
p.shape

torch.Size([3])

In [16]:
P = N.float()

In [10]:
P = (N+1).float()

In [11]:
P.shape

torch.Size([729, 27])

In [12]:
N.shape

torch.Size([729, 27])

In [13]:
P /= P.sum(1, keepdims=True)

In [16]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

mrn.
avr.
lnilyluryigr.
inann.
en.


In [20]:
log_likelihood = 0.0
n = 0

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1] * stoi[ch2]
    ix2 = stoi[ch3]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

print(f'{log_likelihood}')
nll = -log_likelihood
print(f'{nll}')
print(f'{nll/n}')
    

-466903.0
466903.0
2.3807854652404785


*Bigram: log_likelihood=tensor(-564996.8125, grad_fn=<AddBackward0>)
nll=tensor(564996.8125, grad_fn=<NegBackward0>)
2.476470470428467

*trigram: -466903.0  
466903.0  
2.3807854652404785  

*Hence trigram improved*